In [4]:
import sys
sys.path.append('/home/sharedrive/nafi/traffic/lib/python3.9/site-packages')

import pandas as pd
from chronos import ChronosPipeline
import torch
from table_evaluator import TableEvaluator
import matplotlib.pyplot as plt
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import Metadata
import numpy as np
from ctgan import CTGAN
from ctgan import load_demo
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
import xgboost as xgb
from table_evaluator import TableEvaluator
import pickle
import os

In [5]:
itu_test_df = pd.read_csv("/home/sharedrive/nafi/trafficp3/datasets/feature_selected_neg_itu_test.csv")
itu_test_df.shape

(511540, 23)

In [6]:
def load_label_encoder(file_name):
    save_path = "/home/sharedrive/nafi/trafficp3/cicidis"
    with open(f'{save_path}/models/{file_name}_label_encoder.pkl', 'rb') as le_file:
        return pickle.load(le_file)

def load_scaler(file_name):
    save_path = "/home/sharedrive/nafi/trafficp3/cicidis"
    with open(f'{save_path}/models/{file_name}_scaler.pkl', 'rb') as scaler_file:
        return pickle.load(scaler_file)

def load_model(file_name, rounds, num_samples=1000, epochi=1500):
    save_path = "/home/sharedrive/nafi/trafficp3/cicidis"
    with open(f'{save_path}/models/{file_name}sample{num_samples}epoch{epochi}rounds{rounds}DT.pkl', 'rb') as model:
        return pickle.load(model) 


def inference(test_df, load_file_name, rounds, num_sample=1000):
    x_test = test_df.drop(columns=["Label"])
    y_test = test_df["Label"]

    scaler = load_scaler(load_file_name)
    le = load_label_encoder(load_file_name)
    model = load_model(load_file_name, rounds)
    
    x_test = scaler.transform(x_test)
    y_test = le.transform(y_test)
    print("scaler and label encoder loaded & applied!!")


    pred_rf = model.predict(x_test)

    print(f"Classification Report for Sample Size {num_sample}:")
    print(classification_report(y_test, pred_rf, digits=4, target_names=le.classes_))

In [7]:
inference(itu_test_df, 'synthetic_from_cicids_ctgan', 11)


scaler and label encoder loaded & applied!!
Classification Report for Sample Size 1000:
                           precision    recall  f1-score   support

                   BENIGN     0.9837    0.5597    0.7135    410865
                      Bot     0.0067    0.9774    0.0133       354
                     DDoS     0.9606    0.3631    0.5270     23160
            DoS_GoldenEye     0.9639    0.4310    0.5956      1861
                 DoS_Hulk     0.6787    0.3095    0.4251     41626
         DoS_Slowhttptest     0.5733    0.5272    0.5493       994
            DoS_slowloris     0.8151    0.5677    0.6693      1048
              FTP-Patator     0.9099    0.4784    0.6271      1436
               Heartbleed     1.0000    1.0000    1.0000         2
             Infiltration     0.0007    0.6667    0.0014         6
                 PortScan     0.0006    0.0010    0.0008     28728
              SSH-Patator     0.9885    0.0806    0.1490      1067
  Web_Attack__Brute_Force     0.0097    

In [8]:
inference(itu_test_df, 'synthetic_from_cicids_ctgan', 22)

scaler and label encoder loaded & applied!!
Classification Report for Sample Size 1000:
                           precision    recall  f1-score   support

                   BENIGN     0.9624    0.6594    0.7826    410865
                      Bot     0.0032    0.6808    0.0064       354
                     DDoS     0.0198    0.0015    0.0028     23160
            DoS_GoldenEye     0.6947    0.4476    0.5444      1861
                 DoS_Hulk     0.7113    0.5204    0.6011     41626
         DoS_Slowhttptest     0.4344    0.7525    0.5508       994
            DoS_slowloris     0.8441    0.5840    0.6904      1048
              FTP-Patator     0.9120    0.4694    0.6198      1436
               Heartbleed     1.0000    1.0000    1.0000         2
             Infiltration     0.0001    0.6667    0.0002         6
                 PortScan     0.0017    0.0020    0.0018     28728
              SSH-Patator     0.9849    0.4892    0.6537      1067
  Web_Attack__Brute_Force     0.1510    

In [9]:
inference(itu_test_df, 'synthetic_from_cicids_ctgan', 33)

scaler and label encoder loaded & applied!!
Classification Report for Sample Size 1000:
                           precision    recall  f1-score   support

                   BENIGN     0.9958    0.9907    0.9932    410865
                      Bot     0.9231    0.6102    0.7347       354
                     DDoS     0.9995    0.9995    0.9995     23160
            DoS_GoldenEye     0.9908    0.9887    0.9898      1861
                 DoS_Hulk     0.9191    0.9769    0.9471     41626
         DoS_Slowhttptest     0.9820    0.9899    0.9860       994
            DoS_slowloris     0.9981    0.9990    0.9986      1048
              FTP-Patator     0.9965    0.9868    0.9916      1436
               Heartbleed     1.0000    1.0000    1.0000         2
             Infiltration     0.7143    0.8333    0.7692         6
                 PortScan     0.9940    0.9998    0.9969     28728
              SSH-Patator     0.9981    0.4967    0.6633      1067
  Web_Attack__Brute_Force     0.9157    

In [10]:
inference(itu_test_df, 'synthetic_from_cicids_tvae', 11)

scaler and label encoder loaded & applied!!
Classification Report for Sample Size 1000:
                           precision    recall  f1-score   support

                   BENIGN     0.9834    0.7879    0.8748    410865
                      Bot     0.0030    0.5706    0.0060       354
                     DDoS     0.5533    0.9958    0.7114     23160
            DoS_GoldenEye     0.7934    0.9511    0.8651      1861
                 DoS_Hulk     0.4031    0.4228    0.4127     41626
         DoS_Slowhttptest     0.8269    0.8219    0.8244       994
            DoS_slowloris     0.9866    0.9103    0.9469      1048
              FTP-Patator     0.9101    0.4791    0.6277      1436
               Heartbleed     1.0000    1.0000    1.0000         2
             Infiltration     0.0003    0.6667    0.0005         6
                 PortScan     0.7012    0.0040    0.0080     28728
              SSH-Patator     0.2871    0.4911    0.3624      1067
  Web_Attack__Brute_Force     0.0038    

In [11]:
inference(itu_test_df, 'synthetic_from_cicids_tvae', 22)

scaler and label encoder loaded & applied!!
Classification Report for Sample Size 1000:
                           precision    recall  f1-score   support

                   BENIGN     0.9727    0.7633    0.8554    410865
                      Bot     0.0031    0.9859    0.0062       354
                     DDoS     0.5514    0.9930    0.7091     23160
            DoS_GoldenEye     0.8333    0.9592    0.8918      1861
                 DoS_Hulk     0.8368    0.3577    0.5012     41626
         DoS_Slowhttptest     0.4229    0.7173    0.5321       994
            DoS_slowloris     0.8151    0.8034    0.8092      1048
              FTP-Patator     0.9400    0.9923    0.9654      1436
               Heartbleed     1.0000    1.0000    1.0000         2
             Infiltration     0.0072    0.6667    0.0143         6
                 PortScan     0.8904    0.0023    0.0045     28728
              SSH-Patator     0.2277    0.4930    0.3115      1067
  Web_Attack__Brute_Force     0.1172    

In [12]:
inference(itu_test_df, 'synthetic_from_cicids_tvae', 33)

scaler and label encoder loaded & applied!!
Classification Report for Sample Size 1000:
                           precision    recall  f1-score   support

                   BENIGN     0.9958    0.9907    0.9932    410865
                      Bot     0.9149    0.6073    0.7301       354
                     DDoS     0.9994    0.9995    0.9995     23160
            DoS_GoldenEye     0.9903    0.9882    0.9892      1861
                 DoS_Hulk     0.9190    0.9769    0.9471     41626
         DoS_Slowhttptest     0.9830    0.9899    0.9865       994
            DoS_slowloris     0.9952    0.9981    0.9967      1048
              FTP-Patator     0.9965    0.9868    0.9916      1436
               Heartbleed     1.0000    1.0000    1.0000         2
             Infiltration     0.8333    0.8333    0.8333         6
                 PortScan     0.9940    0.9998    0.9969     28728
              SSH-Patator     0.9981    0.4977    0.6642      1067
  Web_Attack__Brute_Force     0.9163    